In [8]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import AdaBoostClassifier
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from statistics import mean
from gensim.models import FastText
from sklearn.metrics.pairwise import cosine_similarity
import json
import numpy as np

In [2]:
doc_to_title = {}
with open('docs_titles.tsv') as f:
    for num_line, line in enumerate(f):
        if num_line == 0:
            continue
        data = line.strip().split('\t', 1)
        doc_id = int(data[0])
        if len(data) == 1:
            title = ''
        else:
            title = data[1]
        doc_to_title[doc_id] = title
print (len(doc_to_title))

28026


In [3]:
import pandas as pd
train_data = pd.read_csv('train_groups.csv')
traingroups_titledata = {}
for i in range(len(train_data)):
    new_doc = train_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    target = new_doc['target']
    title = doc_to_title[doc_id]
    if doc_group not in traingroups_titledata:
        traingroups_titledata[doc_group] = []
    traingroups_titledata[doc_group].append((doc_id, title, target))

In [4]:
vec = {}
with open('titles_stemming.txt', 'r', encoding='utf8') as file:
    vec = json.load(file)
sentences = []
for value in vec.values():
    sentences.append(value)

In [5]:
model_ted = FastText(sentences, size=50, window=5, min_count=5, workers=4, sg=1)

In [43]:
train_data = pd.read_csv('train_groups.csv')
traingroups_titledata = {}
for i in range(len(train_data)):
    new_doc = train_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    target = new_doc['target']
    title = doc_to_title[doc_id]
    if doc_group not in traingroups_titledata:
        traingroups_titledata[doc_group] = []
    traingroups_titledata[doc_group].append((doc_id, title, target))

# Fasttext embeddings
y_train = []
X_train = []
doc_ids = []
X_embed = []
groups_train = []
for new_group in traingroups_titledata:
    docs = traingroups_titledata[new_group]
    for k, (doc_id, title, target_id) in enumerate(docs):
        doc_ids.append(doc_id)
        y_train.append(target_id)
        groups_train.append(new_group)
        all_dist = []
        words = set(title.strip().split())
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j, target_j = docs[j]
            words_j = set(title_j.strip().split())
            all_dist.append(len(words.intersection(words_j)))
        X_train.append(sorted(all_dist, reverse=True)[0:25]    )
        # embeddings
        X_embed.append(model_ted.wv[vec[str(doc_id)]])
X_train = np.array(X_train)
X_embed = np.array(X_embed)
X_train = np.concatenate((X_train, X_embed), axis=1)
y_train = np.array(y_train)
groups_train = np.array(groups_train)

In [24]:
sentences_train = [sentences[i] for i in doc_ids]

In [25]:
model_train = FastText(sentences_train, size=50, window=5, min_count=5, workers=4, sg=1)

In [26]:
train_data = pd.read_csv('train_groups.csv')
traingroups_titledata = {}
for i in range(len(train_data)):
    new_doc = train_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    target = new_doc['target']
    title = doc_to_title[doc_id]
    if doc_group not in traingroups_titledata:
        traingroups_titledata[doc_group] = []
    traingroups_titledata[doc_group].append((doc_id, title, target))

# Fasttext embeddings
y_train = []
X_train = []
X_embed = []
groups_train = []
for new_group in traingroups_titledata:
    docs = traingroups_titledata[new_group]
    for k, (doc_id, title, target_id) in enumerate(docs):
        doc_ids.append(doc_id)
        y_train.append(target_id)
        groups_train.append(new_group)
        all_dist = []
        words = set(title.strip().split())
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j, target_j = docs[j]
            words_j = set(title_j.strip().split())
            all_dist.append(len(words.intersection(words_j)))
        X_train.append(sorted(all_dist, reverse=True)[0:25]    )
        # embeddings
        X_embed.append(model_train.wv[vec[str(doc_id)]])
X_train = np.array(X_train)
X_embed = np.array(X_embed)
X_train = np.concatenate((X_train, X_embed), axis=1)
y_train = np.array(y_train)
groups_train = np.array(groups_train)

In [44]:
test_data = pd.read_csv('test_groups.csv')
testgroups_titledata = {}
for i in range(len(test_data)):
    new_doc = test_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    title = doc_to_title[doc_id]
    if doc_group not in testgroups_titledata:
        testgroups_titledata[doc_group] = []
    testgroups_titledata[doc_group].append((doc_id, title))
X_test = []
doc_ids = []
X_embed = []
groups_test = []
for new_group in testgroups_titledata:
    docs = testgroups_titledata[new_group]
    for k, (doc_id, title) in enumerate(docs):
        doc_ids.append(doc_id)
        groups_test.append(new_group)
        all_dist = []
        words = set(title.strip().split())
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j = docs[j]
            words_j = set(title_j.strip().split())
            all_dist.append(len(words.intersection(words_j)))
        X_test.append(sorted(all_dist, reverse=True)[0:25])
        X_embed.append(model_ted.wv[vec[str(doc_id)]])
X_test = np.array(X_test)
X_embed = np.array(X_embed)
groups_test = np.array(groups_test)
X_test = np.concatenate((X_test, X_embed), axis=1)

In [31]:
doc_ids

[6710,
 4030,
 5561,
 4055,
 4247,
 5983,
 5784,
 4700,
 4093,
 6487,
 7485,
 6705,
 6418,
 6261,
 5360,
 6626,
 6548,
 5010,
 4791,
 4576,
 5746,
 7115,
 5084,
 5600,
 7008,
 6026,
 5398,
 4041,
 5478,
 5004,
 4936,
 4733,
 4119,
 5095,
 6160,
 6777,
 4778,
 6123,
 4517,
 7147,
 5276,
 6187,
 6961,
 4888,
 4556,
 5686,
 5160,
 7001,
 6640,
 7133,
 7407,
 5950,
 7119,
 6542,
 7152,
 4839,
 6897,
 5781,
 6294,
 4876,
 5883,
 5584,
 6684,
 5466,
 4973,
 6963,
 7231,
 4786,
 7190,
 6038,
 6242,
 5401,
 6755,
 6624,
 6473,
 4797,
 7433,
 4428,
 4148,
 5847,
 4121,
 4544,
 4719,
 5677,
 7359,
 5050,
 5615,
 4591,
 5521,
 6078,
 6678,
 7256,
 4603,
 4859,
 5548,
 6049,
 4384,
 4319,
 8578,
 9959,
 8101,
 8213,
 7910,
 10523,
 8092,
 8577,
 9377,
 9219,
 9288,
 10512,
 9391,
 9694,
 8086,
 9190,
 8124,
 9200,
 10115,
 10085,
 9196,
 7990,
 8139,
 9150,
 8496,
 9120,
 10174,
 8041,
 8079,
 11001,
 8512,
 8843,
 9274,
 10148,
 7862,
 9160,
 9849,
 8133,
 7982,
 7907,
 9895,
 9612,
 11162,
 8951

In [33]:
sentences_test = [sentences[i] for i in doc_ids]

IndexError: list index out of range

In [45]:
my_sk = StandardScaler()
X_train_sk = my_sk.fit_transform(X_train)
X_test_sk = my_sk.fit_transform(X_test)

In [46]:
clf_e_1 = RandomForestClassifier(max_depth=10)
clf_e_2 = ExtraTreesClassifier(max_depth=15)
#clf_e_3 = LogisticRegression()
clf_e_4 = xgb.XGBClassifier()

In [47]:
clf_e_1.fit(X_train_sk, y_train)
clf_e_2.fit(X_train_sk, y_train)
#clf_e_3.fit(X_train_sk, y_train)
clf_e_4.fit(X_train_sk, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [48]:
res = (clf_e_1.predict_proba(X_train_sk)[:, 1] + clf_e_2.predict_proba(X_train_sk)[: ,1] + clf_e_4.predict_proba(X_train_sk)[:, 1])/ 3

In [49]:
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(res > th, dtype=int)))

0.6879711221998088
0.7413446768285478
0.811476519150088
0.8301272565847884
0.8246712586098935
0.8098923283983849
0.7227741330834115


In [50]:
f1_score(y_train, np.array(res > 0.4, dtype=int))

0.8301272565847884